<a href="https://colab.research.google.com/github/ML-Challenge/week4-unsupervised-learning/blob/master/L3.Decorrelating%20data%20and%20dimension%20reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" /></a>

Dimension reduction summarizes a dataset using its common occuring patterns. In this lesson, we'll learn about the most fundamental of dimension reduction techniques, "Principal Component Analysis" ("PCA"). PCA is often used before supervised learning to improve model performance and generalization. It can also be useful for unsupervised learning. For example, we'll employ a variant of PCA that will allow us to cluster Wikipedia articles by their content!

# Setup

In [ ]:
# Download lesson datasets
# Required if you're using Google Colab
!wget "https://github.com/ML-Challenge/week4-unsupervised-learning/raw/master/datasets.zip"
!unzip -o datasets.zip

In [ ]:
# Import utils
# We'll be using this module throughout the lesson
import utils

In [ ]:
# Import dependencies
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# and setting the size of all plots.
plt.rcParams['figure.figsize'] = [11, 7]

# Visualizing the PCA transformation

In the next two lessons we'll learn techniques for dimension reduction.

## Dimension reduction

Dimension reduction finds patterns in data, and uses these patterns to re-express it in a compressed form. This makes subsequent computation with the data much more efficient, and this can be a big deal in a world of big datasets. However, the most important function of dimension reduction is to reduce a dataset to its bare bones, discarding noisy features that cause big problems for supervised learning tasks like regression and classification. In many real-world applications, it's dimension reduction that makes prediction possible.

## Principal Component Analysis

In this lesson, we'll learn about the most fundamental of dimension reduction techniques. It's called Principal Component Analysis, or `PCA` for short. PCA performs dimension reduction in two steps, and the first one, called de-correlation, doesn't change the dimension of the data at all. It's this first step that we'll focus on next.

In this first step, PCA rotates the samples so that they are aligned with the coordinate axes. In fact, it does more than this: PCA also shifts the samples so that they have mean zero. These scatter plots show the effect of PCA applied to two features of the wine dataset.

![PCA](assets/3-1.png)

Notice that no information is lost - this is true no matter how many features our dataset has.

`scikit-learn` has an implementation of PCA, and it has fit and transform methods just like StandardScaler: 

* The `fit` method learns how to shift and how to rotate the samples, but doesn't actually change them. 
* The `transform` method, on the other hand, applies the transformation that fit learned. In particular, the transform method can be applied to new, unseen samples.

## Using scikit-learn PCA

Let's see PCA in action on the some features of the wine dataset. Firstly, import PCA.

```
from sklearn.decomposition import PCA
```

Now create a PCA object, and fit it to the samples.

```
model = PCA()
model.fit(utils.wine)
```

Then use the fit PCA object to transform the samples. This returns a new array of transformed samples.

```
transformed = model.transform(utils.wine)
```

## PCA features

This new array has the same number of rows and columns as the original sample array. In particular, there is one row for each transformed sample. The columns of the new array correspond to `PCA features`, just as the original features corresponded to columns of the original array.

It is often the case that the features of a dataset are correlated. This is the case with many of the features of the wine dataset, for instance. However, PCA, due to the rotation it performs, de-correlates the data, in the sense that the columns of the transformed array are not linearly correlated.

## Pearson correlation

Linear correlation can be measured with the Pearson correlation. It takes values between -1 and 1, where larger values indicate a stronger correlation, and 0 indicates no linear correlation. Here are some examples of features with varying degrees of correlation.

![Decorrelated](assets/3-2.png)

## Principal components

Finally, PCA is called principal component analysis because it learns the principal components of the data. These are the directions in which the samples vary the most, depicted here in red. 

![PCs](assets/3-3.png)

It is the principal components that PCA aligns with the co-ordinate axes. After a PCA model has been fit, the principal components are available as the `components_` attribute. This is numpy array with one row for each principal component.

## Correlated data in nature

We have an array of grains giving the width and length of samples of grain. We suspect that width and length will be correlated. To confirm this, let's make a scatter plot of width vs length and measure their Pearson correlation.

In [ ]:
# Perform the necessary imports
from scipy.stats import pearsonr

In [ ]:
# Assign the 0th column of grains: width
width = utils.grains[:,0]

In [ ]:
# Assign the 1st column of grains: length
length = utils.grains[:,1]

In [ ]:
# Scatter plot width vs length
plt.scatter(width, length)
plt.axis('equal')
plt.show()

In [ ]:
# Calculate the Pearson correlation
correlation, pvalue = pearsonr(width, length)

In [ ]:
# Display the correlation
print(correlation)

As we would expect, the width and length of the grain samples are highly correlated.

## Decorrelating the grain measurements with PCA

We observed in the previous example that the width and length measurements of the grain are correlated. Now, we'll use PCA to decorrelate these measurements, then plot the decorrelated points and measure their Pearson correlation.

In [ ]:
# Import PCA
from sklearn.decomposition import PCA

In [ ]:
# Create PCA instance: model
model = PCA()

In [ ]:
# Apply the fit_transform method of model to grains: pca_features
pca_features = model.fit_transform(utils.grains)

In [ ]:
# Assign 0th column of pca_features: xs
xs = pca_features[:,0]

# Assign 1st column of pca_features: ys
ys = pca_features[:,1]

In [ ]:
# Scatter plot xs vs ys
plt.scatter(xs, ys)
plt.axis('equal')
plt.show()

In [ ]:
# Calculate the Pearson correlation of xs and ys
correlation, pvalue = pearsonr(xs, ys)

# Display the correlation
print(correlation)

Excellent! We've successfully decorrelated the grain measurements with PCA!

# Intrinsic dimension

Consider a dataset with two features: `latitude` and `longitude`. These two features might track the flight of an airplane, for example. This dataset is two dimensional, yet it turns out that it can be closely approximated using only one feature: the displacement along the flight path. This dataset is intrinsically one dimensional.

![Flight Path](assets/3-4.png)

The intrinsic dimension of a dataset is the number of features required to approximate it. The intrinsic dimension informs dimension reduction, because it tells us how much a dataset can be compressed.

Next, we'll gain a solid understanding of the intrinsic dimension, and be able to use PCA to identify it in real-world datasets that have thousands of features.

To better illustrate the intrinsic dimension, let's consider an example dataset containing only some of the samples from the iris dataset. Specifically, let's take three measurements from the iris versicolor samples: `sepal length`, `sepal width`, and `petal width`. So each sample is represented as a point in 3-dimensional space.

However, if we make a 3d scatter plot of the samples, we see that they all lie very close to a flat, 2-dimensional sheet. This means that the data can be approximated by using only two co-ordinates, without losing much information. So this dataset has intrinsic dimension 2.

![3D Scatter](assets/3-5.png)

But scatter plots are only possible if there are 3 features or less. So how can the intrinsic dimension be identified, even if there are many features?

This is where PCA is really helpful. The intrinsic dimension can be identified by counting the PCA features that have high variance. To see how, let's see what happens when PCA is applied to the dataset of versicolor samples.

![PCA 3D](assets/3-6.png)

PCA rotates and shifts the samples to align them with the co-ordinate axes. This expresses the samples using three PCA features.

The PCA features are in a special order. Here is a bar graph showing the variance of each of the PCA features.

![PCA Bar](assets/3-7.png)

As we can see, each PCA feature has less variance than the last, and in this case the last PCA feature has very low variance. This agrees with the scatter plot of the PCA features, where the samples don't vary much in the vertical direction. In the other two directions, however, the variance is apparent.

The intrinsic dimension is the number of PCA features that have significant variance. In our example, only the first two PCA features have significant variance. So this dataset has intrinsic dimension 2, which agrees with what we observed when inspecting the scatter plot.

![PCA Reduction](assets/3-8.png)

Let's see how to plot the variances of the PCA features in practice. Firstly, make the necessary imports.

```
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
```

Then create a PCA model, and fit it to the samples.

```
pca = PCA()
pca.fit(samples)
```

Now create a range enumerating the PCA features,

```
features = range(pca.n_components_)
```

and make a bar plot of the variances; the variances are available as the `explained_variance_` attribute of the PCA model.

```
plt.bar(features, pca.explained_variance_)

plt.xticks(features)
plt.ylabel('variance')
plt.xlabel('PCA feature')

plt.show()
```

The intrinsic dimension is a useful idea that helps to guide dimension reduction. However, it is not always unambiguous. Here is a graph of the variances of the PCA features for the wine dataset.

![Wine PCA](assets/3-9.png)

We could argue for an intrinsic dimension of 2, of 3, or even more, depending upon the threshold we chose. In the next section, we'll learn to use the intrinsic dimension for dimension reduction.

## The first principal component

The first principal component of the data is the direction in which the data varies the most. In this example, our job is to use PCA to find the first principal component of the length and width measurements of the grain samples, and represent it as an arrow on the scatter plot.

In [ ]:
# Make a scatter plot of the untransformed points
plt.scatter(width, length)

# Create a PCA instance: model
model = PCA()

# Fit model to points
model.fit(utils.grains)

# Get the mean of the grain samples: mean
mean = model.mean_

# Get the first principal component: first_pc
first_pc = model.components_[0,:]

# Plot first_pc as an arrow, starting at mean
plt.arrow(mean[0], mean[1], first_pc[0], first_pc[1], color='red', width=0.01)

# Keep axes on same scale
#plt.axis('equal')
plt.show()

This is the direction in which the grain data varies the most.

## Variance of the PCA features

The fish dataset is 6-dimensional. But what is its intrinsic dimension? Let's make a plot of the variances of the PCA features to find out. As before, `fish_samples` is a 2D array, where each row represents a fish. We'll need to standardize the features first.

In [ ]:
# Perform the necessary imports
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
# Create scaler: scaler
scaler = StandardScaler()

In [ ]:
# Create a PCA instance: pca
pca = PCA()

In [ ]:
# Create pipeline: pipeline
pipeline = make_pipeline(scaler, pca)

In [ ]:
# Fit the pipeline to 'samples'
pipeline.fit(utils.fish_samples)

In [ ]:
# Plot the explained variances
features = range(pca.n_components_)
plt.bar(features, pca.explained_variance_)
plt.xlabel('PCA feature')
plt.ylabel('variance')
plt.xticks(features)
plt.show()

 It looks like PCA features 0 and 1 have significant variance.

# Dimension reduction with PCA

Dimension reduction represents the same data using less features and is vital for building machine learning pipelines using real-world data. Finally, in this lesson, we'll learn how to perform dimension reduction using PCA.

We've seen already that the PCA features are in decreasing order of variance. PCA performs dimension reduction by discarding the PCA features with lower variance, which it assumes to be noise, and retaining the higher variance PCA features, which it assumes to be informative.

To use PCA for dimension reduction, we need to specify how many PCA features to keep. For example, specifying `n_components=2` when creating a PCA model tells it to keep only the first two PCA features. A good choice is the intrinsic dimension of the dataset, if we know it. Let's consider the following example.

The iris dataset has 4 features representing the 4 measurements. Here, the measurements are in a numpy array called `samples`. Let's use PCA to reduce the dimension of the iris dataset to only 2. Begin by importing PCA as usual.

```
from sklearn.decomposition import PCA
```

Create a PCA model specifying `n_components=2`, and then fit the model and transform the samples as usual.

```
pca = PCA(n_components=2)

pca.fit(samples)
transformed = pca.transform(samples)
```

Here is a scatterplot of the two PCA features, where the colors represent the three species of iris.

![Iris PCA](assets/3-14.png)

Remarkably, despite having reduced the dimension from 4 to 2, the species can still be distinguished. Remember that PCA didn't even know that there were distinct species. PCA simply took the 2 PCA features with highest variance. As we can see, these two features are very informative.

PCA discards the low variance features, and assumes that the higher variance features are informative. Like all assumptions, there are cases where this doesn't hold. As we saw with the iris dataset, however, it often does in practice.

## Word frequency arrays

In some cases, an alternative implementation of PCA needs to be used. Word frequency arrays are a great example. In a word-frequency array, each row corresponds to a document, and each column corresponds to a word from a fixed vocabulary. The entries of the word-frequency array measure how often each word appears in each document.

![Word Frequency Array](assets/3-13.png)

Only some of the words from the vocabulary appear in any one document, so most entries of the word frequency array are zero. Arrays like this are said to be sparse, and are often represented using a special type of array called a `csr_matrix`.

`csr_matrices` save space by remembering only the non-zero entries of the array. Scikit-learn's PCA doesn't support csr_matrices, and we'll need to use `TruncatedSVD` instead. `TruncatedSVD` performs the same transformation as PCA, but accepts csr matrices as input. Other than that, we interact with `TruncatedSVD` and PCA in exactly the same way.

## Dimension reduction of the fish measurements

In the previous example, we saw that 2 was a reasonable choice for the "intrinsic dimension" of the fish measurements. Now let's use PCA for dimensionality reduction of the fish measurements, retaining only the 2 most important components.

In [ ]:
# Create a PCA model with 2 components: pca
pca = PCA(n_components=2)

In [ ]:
# Create pipeline: pipeline
pipeline = make_pipeline(scaler, pca)

In [ ]:
# Fit the pipeline to 'samples'
pipeline.fit(utils.fish_samples)

In [ ]:
# Transform the scaled samples: pca_features
pca_features = pipeline.transform(utils.fish_samples)

In [ ]:
# Print the shape of pca_features
print(utils.fish_samples.shape)
print(pca_features.shape)

Superb! We've successfully reduced the dimensionality from 6 to 2.

## A tf-idf word-frequency array

In this example, we'll create a tf-idf word frequency array for a toy collection of documents. For this, we use the `TfidfVectorizer` from sklearn. It transforms a list of documents into a word frequency array, which it outputs as a `csr_matrix`. It has `fit()` and `transform()` methods like other sklearn objects.

In [ ]:
documents = ['cats say meow', 'dogs say woof', 'dogs chase cats']

In [ ]:
# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Create a TfidfVectorizer: tfidf
tfidf = TfidfVectorizer() 

In [ ]:
# Apply fit_transform to document: csr_mat
csr_mat = tfidf.fit_transform(documents)

In [ ]:
# Print result of toarray() method
print(csr_mat.toarray())

In [ ]:
# Get the words: words
words = tfidf.get_feature_names()

# Print words
print(words)

## Clustering Wikipedia part I

We saw that `TruncatedSVD` is able to perform PCA on sparse arrays in csr_matrix format, such as word-frequency arrays. Let's combine our knowledge of TruncatedSVD and k-means to cluster some popular pages from Wikipedia. In this example, we build the pipeline. In the next example, we'll apply it to the word-frequency array of some Wikipedia articles.

Let's create a Pipeline object consisting of a TruncatedSVD followed by KMeans. The word-frequency matrix is already computed, so there's no need for a `TfidfVectorizer`).

The Wikipedia dataset we'll be working with was obtained from [here](https://blog.lateral.io/2015/06/the-unknown-perils-of-mining-wikipedia/).

In [ ]:
# Perform the necessary imports
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans

In [ ]:
# Create a TruncatedSVD instance: svd
svd = TruncatedSVD(n_components=50)

In [ ]:
# Create a KMeans instance: kmeans
kmeans = KMeans(n_clusters=6)

In [ ]:
# Create a pipeline: pipeline
pipeline = make_pipeline(svd, kmeans)

## Clustering Wikipedia part II

It is now time to put our pipeline from the previous example to work! We have an array `articles` of tf-idf word-frequencies of some popular Wikipedia articles, and a list `titles` of their titles. Let's use the pipeline to cluster the Wikipedia articles.

In [ ]:
# Fit the pipeline to articles
pipeline.fit(utils.articles)

In [ ]:
# Calculate the cluster labels: labels
labels = pipeline.predict(utils.articles)

In [ ]:
# Create a DataFrame aligning labels and titles: df
df = pd.DataFrame({'label': labels, 'article': utils.titles})

In [ ]:
# Display df sorted by cluster label
print(df.sort_values('label'))

---
**[Week 4 - Unsupervised Learning](https://radu-enuca.gitbook.io/ml-challenge/unsupervised-learning)**

*Have questions or comments? Visit the ML Challenge Mattermost Channel.*